In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as logprob
import scipy.stats as si
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report

In [2]:
# read cleaned csv into dataframe
df = pd.read_csv('Customer-Churn-Records-clean.csv')

# preview Dataframe
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [3]:
# examine target variable distribution -- establish if have a sampling bias
df['Exited'].value_counts()

0    7962
1    2038
Name: Exited, dtype: int64

In [4]:
# return list of columns in dataframe
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Complain', 'Satisfaction Score', 'Card Type',
       'Point Earned'],
      dtype='object')

In [5]:
# create separate DataFrame with NUMERIC columns only, drop categorical columns
df_numeric = df.copy()
df_numeric.drop(['Geography', 'Gender', 'Card Type','HasCrCard','Complain', 'IsActiveMember'], axis=1,inplace=True)

# preview columns dropped as intended
df_numeric.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Exited,Satisfaction Score,Point Earned
0,619,42,2,0.00,1,101348.88,1,2,464
1,608,41,1,83807.86,1,112542.58,0,3,456
2,502,42,8,159660.80,3,113931.57,1,3,377
3,699,39,1,0.00,2,93826.63,0,5,350
4,850,43,2,125510.82,1,79084.10,0,5,425


In [6]:
# Create dataframe for dependent variables
y = df_numeric.loc[:, df_numeric.columns == 'Exited']

# # Create dataframe for independent variable
X = df_numeric.loc[:, df_numeric.columns != 'Exited']

In [7]:
# implement SMOTE to account for imbalanced dataset
os = SMOTE(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, stratify=y)
os_X, os_y = os.fit_resample(X_train, y_train)

In [8]:
# display shape of train, test data
display(X_train.shape)
display(X_test.shape)
display(os_X.shape)

(8000, 8)

(2000, 8)

(12740, 8)

In [9]:
# confirm distribution of dependent variable for oversampling
os_y.value_counts()

Exited
0         6370
1         6370
dtype: int64

In [10]:
# # # Instantiate the random oversampler model
# ros = RandomOverSampler(random_state=42)

# # Fit the original training data to the random_oversampler model
# ros_X, ros_y = ros.fit_resample(X_train, y_train)

In [11]:
# Create variable list for RFE (recursive feature elimination)
df_vars = df_numeric.columns.values.tolist()
y = ['Exited']
X = [i for i in df_vars if i not in y]

In [12]:
# Run recursive feature elimination algorithm to identify which if any feaures can be dropped
selector =  RFE(estimator=LogisticRegression(random_state = 42), n_features_to_select=8, step = 1)

# Fit the RFE model and estimator on the selected features.
selector = selector.fit(os_X, os_y.values.ravel())

# display feature ranking -- most important features are assigned a rank of 1
print(selector.ranking_)

# display mask of selected features
print(selector.support_)

[1 1 1 1 1 1 1 1]
[ True  True  True  True  True  True  True  True]


In [13]:
# Define variables to use to fit probit model
cols=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary', 'Satisfaction Score', 'Point Earned']
X=os_X[cols]
y=os_y['Exited']

In [14]:
# Implement the probit model
probit=logprob.Probit(y,X)

result=probit.fit()

# return results
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.602305
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                 Exited   No. Observations:                12740
Model:                         Probit   Df Residuals:                    12732
Method:                           MLE   Df Model:                            7
Date:                Sun, 04 Jun 2023   Pseudo R-squ.:                  0.1311
Time:                        23:03:21   Log-Likelihood:                -7673.4
converged:                       True   LL-Null:                       -8830.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
CreditScore           -0.0011   8.99e-05    -12.057      0.000      -0.001      -0.001
Age  

In [15]:
# update independent variables based on p-values
colss=['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'Satisfaction Score', 'Point Earned']
X_v2=os_X[colss]

In [16]:
# rerun probit model now that EstimatedSalary is dropped
probit=logprob.Probit(y,X_v2)

result=probit.fit()

# return results
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.602460
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                 Exited   No. Observations:                12740
Model:                         Probit   Df Residuals:                    12733
Method:                           MLE   Df Model:                            6
Date:                Sun, 04 Jun 2023   Pseudo R-squ.:                  0.1308
Time:                        23:03:47   Log-Likelihood:                -7675.3
converged:                       True   LL-Null:                       -8830.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
CreditScore           -0.0011   8.89e-05    -11.896      0.000      -0.001      -0.001
Age  

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
probit=logprob.Probit(y_train,X_train)
probit.fit()
print(probit.fit().summary())

Optimization terminated successfully.
         Current function value: 0.602461
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.602461
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                 Exited   No. Observations:                10192
Model:                         Probit   Df Residuals:                    10184
Method:                           MLE   Df Model:                            7
Date:                Sun, 04 Jun 2023   Pseudo R-squ.:                  0.1308
Time:                        23:05:14   Log-Likelihood:                -6140.3
converged:                       True   LL-Null:                       -7064.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

In [19]:
# extract coefficients for oversampling and store in dataframe
params = pd.DataFrame(probit.fit().params,columns=['coef'],)
params

Optimization terminated successfully.
         Current function value: 0.602461
         Iterations 5


,coef
CreditScore,-1.041322e-03
Age,4.060938e-02
Tenure,-2.587735e-02
Balance,1.959905e-06
NumOfProducts,-4.462342e-01
EstimatedSalary,3.882014e-07
Satisfaction Score,-1.089771e-01
Point Earned,-1.875144e-04


In [21]:
## DISCUSS WITH TEAM ABOUT OVERFITING AND RUNNING MODEL

# predict results for test data
result_mvr = X_test
result_mvr['y_pred'] = result_mvr['CreditScore'] * params['coef'][0] + result_mvr['Age'] * params['coef'][1] + result_mvr['Tenure'] * params['coef'][2] + result_mvr['Balance'] * params['coef'][3] + result_mvr['NumOfProducts'] * params['coef'][4] +  result_mvr['Satisfaction Score'] * params['coef'][5] +  result_mvr['Satisfaction Score'] * params['coef'][6] + result_mvr['Point Earned'] * params['coef'][7] 
result_mvr

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Satisfaction Score,Point Earned,y_pred
4755,512,30,6,0.000000,2,88827.310000,2,471,-0.668881
8137,675,35,4,92864.219572,1,198159.995910,4,627,-0.202781
7468,841,36,5,156021.310000,1,122662.980000,4,242,-0.164934
10339,706,47,6,105086.279996,1,180344.581321,5,743,0.093721
8850,584,36,6,137690.460673,1,151774.835120,3,718,0.060602
...,...,...,...,...,...,...,...,...,...
11039,785,36,1,103561.572376,1,62572.905182,3,958,-0.131210
3603,678,66,8,0.000000,2,47117.030000,1,636,0.646480
4977,850,34,2,0.000000,2,171706.660000,4,589,-0.994981
2333,547,22,7,141287.150000,1,118142.790000,5,282,-0.624426


In [22]:
# function returns probability that a random variable takes on a value less than parameter passed
def normsdist(z):
    z = si.norm.cdf(z,0.0,1.0)
    return (z)

# calculate probability that a random variables takes on a value less than 1.645
normsdist(1.645)

0.9500150944608786

In [23]:
# calculate probit value for y_pred
result_mvr['y_pred_Probit'] = normsdist(result_mvr['y_pred'])
result_mvr

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Satisfaction Score,Point Earned,y_pred,y_pred_Probit
4755,512,30,6,0.000000,2,88827.310000,2,471,-0.668881,0.251786
8137,675,35,4,92864.219572,1,198159.995910,4,627,-0.202781,0.419653
7468,841,36,5,156021.310000,1,122662.980000,4,242,-0.164934,0.434498
10339,706,47,6,105086.279996,1,180344.581321,5,743,0.093721,0.537335
8850,584,36,6,137690.460673,1,151774.835120,3,718,0.060602,0.524162
...,...,...,...,...,...,...,...,...,...,...
11039,785,36,1,103561.572376,1,62572.905182,3,958,-0.131210,0.447805
3603,678,66,8,0.000000,2,47117.030000,1,636,0.646480,0.741016
4977,850,34,2,0.000000,2,171706.660000,4,589,-0.994981,0.159873
2333,547,22,7,141287.150000,1,118142.790000,5,282,-0.624426,0.266174


In [24]:
# create a dictionary of the predicted probit values
d = {'y_pred_proba': result_mvr['y_pred_Probit']}

# convert dictionary to dataframe
df_ypred = pd.DataFrame(data = d)

# reset index on dataFrame
df_ypred = df_ypred.reset_index()

# drop index dataframe
df_ypred.drop(['index'], axis=1, inplace = True)

# create place holder column for calculated y-pred values (logic applied to ypred proba)
df_ypred['y_pred'] = 0.000

# display dataframe
df_ypred.head()

,y_pred_proba,y_pred
0,0.251786,0.0
1,0.419653,0.0
2,0.434498,0.0
3,0.537335,0.0
4,0.524162,0.0


In [25]:
# apply logic to each y_pred_proba to generate binary value for predicted outcome
# since training model had 50/50 split in sampled data; 50% is the threshold to determine if y_pred proba should be a 0 or 1
# if y_pred_proba > .5 value is replaced with 1, else 0

for i in range(0,len(df_ypred['y_pred_proba'])):
    if df_ypred['y_pred_proba'][i] > 0.500:
        df_ypred['y_pred'][i] = 1.000
    else: 
        df_ypred['y_pred'][i] = 0.000
        
df_ypred.head()

,y_pred_proba,y_pred
0,0.251786,0.0
1,0.419653,0.0
2,0.434498,0.0
3,0.537335,1.0
4,0.524162,1.0


In [26]:
# create np array from y_pred column
y_pred = np.array(df_ypred['y_pred'])

# convert to integer
y_pred = y_pred.astype('int64')

# preview array
y_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [27]:
print('Accuracy of Probit Model on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

Accuracy of Probit Model on test set: 0.69


In [28]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[924 350]
 [432 842]]


In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.73      0.70      1274
           1       0.71      0.66      0.68      1274

    accuracy                           0.69      2548
   macro avg       0.69      0.69      0.69      2548
weighted avg       0.69      0.69      0.69      2548



In [30]:
y_test.value_counts()

0    1274
1    1274
Name: Exited, dtype: int64